In [1]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer

In [2]:
def getInputNumbers():
    '''
    displays two input boxes for taking user input. Performs validation on the input.
    
    returns: the the two input numbers
    '''
    #input validation code. Nothing other than 0 and 1 must be allowed, and length check
    isValid = False

    while not isValid:
        num1 = input("Enter a binary number containing maximum 3 bits").strip()
        num2 = input("Enter another binary number containing maximum 3 bits").strip()
        try:
            if len(num1)<=3 and len(num2)<=3:
                isValid=True
            decimal_num1 = int(num1,2)
            decimal_num2 = int(num2,2)
        except:
            isValid=False
    return num1,num2


In [3]:
def initializeQuantumRegisters(q_num1,q_num2,bin_number1,bin_number2,qc_add):
    '''
    q_num1: quantum register to hold bin_number1
    q_num2: quantum register to hold bin_number2
    qc_add: Quantum circuit that contains q_num1 and q_num2 as two of its quantum registers
    
    When we intialize a quantum register with n qubits, all qubits are initialized to |0> state.
    This function initializes q_num1, q_num2 according to the input numbers bin_number1, bin_number2
    using quantum X logic gate to flip the required qubits to |1> state.
    X gate is similar to classical NOT gate
    
    returns: None
    '''
    #initializing the quatum register qubits according to the classical num1 and num2 input
    # we are iterating the classical value from the most significant bit to least.
    #Example, if you entered the num1 value as "00011101", then index 0, num1[0] returns 0 and num1[7]=1
    # read the index 0 of num1 and assign the value to index maximum, 7(use length of number) of the quantum register.
    # Note that second q register is 1 qubit longer, we leave the most significat qubit of this with 
    #its inital value zero, as is

    len_num1 = len(bin_number1) 
    len_num2 = len(bin_number2)
    for i in range(len_num1):
        if bin_number1[i] == "1":
            qc_add.x(q_num1[len_num1 - (i+1)]) #Flip the qubit from 0 to 1
    for i in range(len_num2):
        if bin_number2[i] == "1":
            qc_add.x(q_num2[len_num2 - (i+1)]) #Flip the qubit from 0 to 1

In [4]:
def quantumAdd(bin_number1, bin_number2):
    '''
        bin_number1: binary number
        bin_number2: binary number
        performs addition using quantum circuit, uses X, CX and CCX gates
        returns: the sum of bin_number1 and bin_number2
    '''
    
    #Figure out length n of the input numbers. We will need to intialize the quantum registers
    #with number of qubits equal to the max length of(num1, num2), and another quantum register
    #with n+1 qubits to hold the sum, the plus 1 to handle carry bit'''

    n = len(bin_number1)
    if n<len(bin_number2):
        n= len(bin_number2)
        
    #Initializing the registers:
    # Quantum registers for holding the numbers to be added, sum and carry qubits
    # Classical register to hold the measurement of the sum read from quantum register.

    q_num1 = QuantumRegister(n) #First number
    q_num2 = QuantumRegister(n+1) #Second number, then sum
    q_carry = QuantumRegister(n) #Carry bits
    c_sum = ClassicalRegister(n+1) #Classical output

    #Combining all of them into one quantum circuit
    qc_add = QuantumCircuit(q_num1, q_num2, q_carry, c_sum)
    
    #re-initialize quantum registers according to the bit values in the input numbers provided
    initializeQuantumRegisters(q_num1,q_num2,bin_number1,bin_number2,qc_add)
    
    #perform the quantum addition using cx and ccx gates
    for index in range(n):
        if index<n-1:
            qc_add.ccx(q_num1[index], q_num2[index],q_carry[index+1])
            qc_add.cx(q_num1[index], q_num2[index])

            qc_add.ccx(q_carry[index],q_num2[index], q_carry[index+1])
            qc_add.cx(q_carry[index], q_num2[index])

        else:
            qc_add.ccx(q_num1[index], q_num2[index],q_num2[index+1])
            qc_add.cx(q_num1[index], q_num2[index])

            qc_add.ccx(q_carry[index],q_num2[index], q_num2[index+1])
            qc_add.cx(q_carry[index], q_num2[index])
    # measure the contents of q_num2 which should hold the sum
    #Measure qubits and store results in classical register c_sum
    for i in range(n+1):
        qc_add.measure(q_num2[i], c_sum[i])
    
    #Executing the quantum program 5 times:

    job = execute(qc_add,Aer.get_backend('qasm_simulator'),shots=5)
    #The results can be accessed as a dictionary:
    counts = job.result().get_counts(qc_add)  
    #print(num1, "\n + ",num2,"\n is = \n\n")
    #print(counts)
    return list(counts.keys())[0]